In [2]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

sc = SparkSession.builder.appName("word-count").master("local").getOrCreate()
sc
sc.conf.set("spark.sql.shuffle.partitions", 5)

In [5]:
df = sc.read.json("device_01.json")
df.show(truncate = False)
df.printSchema()

+----------+------------------------------------------------+------------------------------------+-----------+--------------+--------------------------+
|customerId|data                                            |eventId                             |eventOffset|eventPublisher|eventTime                 |
+----------+------------------------------------------------+------------------------------------+-----------+--------------+--------------------------+
|CI00103   |{[{D001, C, ERROR, 15}, {D002, C, SUCCESS, 16}]}|e3cb26d3-41b2-49a2-84f3-0156ed8d7502|10001      |device        |2023-01-05 11:13:53.643364|
+----------+------------------------------------------------+------------------------------------+-----------+--------------+--------------------------+

root
 |-- customerId: string (nullable = true)
 |-- data: struct (nullable = true)
 |    |-- devices: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- deviceId: string (nullable = true)


In [7]:
from pyspark.sql.functions import *
mid_flat_df = df.withColumn("devices", explode(col("data.devices"))).drop(col("data"))
mid_flat_df.printSchema()

root
 |-- customerId: string (nullable = true)
 |-- eventId: string (nullable = true)
 |-- eventOffset: long (nullable = true)
 |-- eventPublisher: string (nullable = true)
 |-- eventTime: string (nullable = true)
 |-- devices: struct (nullable = true)
 |    |-- deviceId: string (nullable = true)
 |    |-- measure: string (nullable = true)
 |    |-- status: string (nullable = true)
 |    |-- temperature: long (nullable = true)



In [9]:
flat_df =( mid_flat_df
            .withColumn("deviceId", col("devices.deviceId"))
            .withColumn("measure", col("devices.measure"))
            .withColumn("status", col("devices.status"))
            .withColumn("temperature", col("devices.temperature"))
            .drop("devices")
)
flat_df.printSchema()
flat_df.show()

root
 |-- customerId: string (nullable = true)
 |-- eventId: string (nullable = true)
 |-- eventOffset: long (nullable = true)
 |-- eventPublisher: string (nullable = true)
 |-- eventTime: string (nullable = true)
 |-- deviceId: string (nullable = true)
 |-- measure: string (nullable = true)
 |-- status: string (nullable = true)
 |-- temperature: long (nullable = true)

+----------+--------------------+-----------+--------------+--------------------+--------+-------+-------+-----------+
|customerId|             eventId|eventOffset|eventPublisher|           eventTime|deviceId|measure| status|temperature|
+----------+--------------------+-----------+--------------+--------------------+--------+-------+-------+-----------+
|   CI00103|e3cb26d3-41b2-49a...|      10001|        device|2023-01-05 11:13:...|    D001|      C|  ERROR|         15|
|   CI00103|e3cb26d3-41b2-49a...|      10001|        device|2023-01-05 11:13:...|    D002|      C|SUCCESS|         16|
+----------+--------------------